# 🌸 Análisis de Perfumes Fragantica - EDA

## 📖 Introducción
Este cuaderno presenta un Análisis Exploratorio de Datos (EDA) del dataset de Fragantica. El objetivo es descubrir tendencias en la industria del perfume, analizar el rendimiento de las marcas, explorar las notas olfativas y comprender las preferencias de los usuarios.

El dataset incluye información sobre nombres de perfumes, marcas, países, puntuaciones, años de lanzamiento y perfiles olfativos detallados (notas y acordes).

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Configurar tema de Plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

# Opciones de visualización
pd.set_option('display.max_columns', None)

## 📥 1. Carga de Datos e Inspección Inicial

In [2]:
# Cargar el dataset
file_path = 'fra_cleaned.csv'
try:
    df = pd.read_csv(file_path, sep=';', decimal=',', encoding='latin-1')
    print("¡Dataset cargado con éxito!")
except Exception as e:
    print(f"Error al cargar el dataset: {e}")

¡Dataset cargado con éxito!


In [3]:
# Mostrar las primeras filas
df.head()

,url,Perfume,Brand,Country,Gender,Rating Value,Rating Count,Year,Top,Middle,Base,Perfumer1,Perfumer2,mainaccord1,mainaccord2,mainaccord3,mainaccord4,mainaccord5
0,https://www.fragrantica.com/perfume/xerjoff/ac...,accento-overdose-pride-edition,xerjoff,Italy,unisex,1.42,201,2022.0,"fruity notes, aldehydes, green notes","bulgarian rose, egyptian jasmine, lily-of-the-...","eucalyptus, pine",unknown,NaN,rose,woody,fruity,aromatic,floral
1,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2024,jean-paul-gaultier,France,women,1.86,70,2024.0,"yuzu, citruses","orange blossom, neroli","musk, blonde woods",unknown,NaN,citrus,white floral,sweet,fresh,musky
2,https://www.fragrantica.com/perfume/jean-paul-...,classique-pride-2023,jean-paul-gaultier,France,unisex,1.91,285,2023.0,"blood orange, yuzu","neroli, orange blossom","musk, white woods",natalie gracia-cetto,quentin bisch,citrus,white floral,sweet,fresh spicy,musky
3,https://www.fragrantica.com/perfume/bruno-bana...,pride-edition-man,bruno-banani,Germany,men,1.92,59,2019.0,"guarana, grapefruit, red apple","walnut, lavender, guava","vetiver, benzoin, amber",unknown,NaN,fruity,nutty,woody,tropical,NaN
4,https://www.fragrantica.com/perfume/jean-paul-...,le-male-pride-collector,jean-paul-gaultier,France,men,1.93,632,2020.0,"mint, lavender, cardamom, artemisia, bergamot","caraway, cinnamon, orange blossom","vanilla, sandalwood, amber, cedar, tonka bean",francis kurkdjian,NaN,aromatic,warm spicy,fresh spicy,cinnamon,vanilla


In [4]:
# Información básica
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24063 entries, 0 to 24062
Data columns (total 18 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   url           24063 non-null  object 
 1   Perfume       24063 non-null  object 
 2   Brand         24063 non-null  object 
 3   Country       24063 non-null  object 
 4   Gender        24063 non-null  object 
 5   Rating Value  24063 non-null  float64
 6   Rating Count  24063 non-null  int64  
 7   Year          22026 non-null  float64
 8   Top           24063 non-null  object 
 9   Middle        24063 non-null  object 
 10  Base          24063 non-null  object 
 11  Perfumer1     24063 non-null  object 
 12  Perfumer2     1336 non-null   object 
 13  mainaccord1   24063 non-null  object 
 14  mainaccord2   24050 non-null  object 
 15  mainaccord3   23949 non-null  object 
 16  mainaccord4   23675 non-null  object 
 17  mainaccord5   23082 non-null  object 
dtypes: float64(2), int64(1), o

In [5]:
# Estadísticas descriptivas
df.describe()

,Rating Value,Rating Count,Year
count,24063.000000,24063.000000,22026.000000
mean,3.960379,501.396542,2012.455961
std,0.277429,1429.484690,13.526737
min,1.420000,26.000000,1781.000000
25%,3.790000,56.000000,2010.000000
50%,3.970000,127.000000,2015.000000
75%,4.150000,360.000000,2019.000000
max,4.930000,29858.000000,2024.000000


In [6]:
# Comprobar valores nulos
missing_values = df.isnull().sum()
missing_values[missing_values > 0]

,0
Year,2037
Perfumer2,22727
mainaccord2,13
mainaccord3,114
mainaccord4,388
mainaccord5,981


## 📊 2. Análisis Univariante

### ⭐ 2.1 Distribución de las Puntuaciones

In [7]:
# Histograma del Valor de la Puntuación
fig = px.histogram(df, x='Rating Value', nbins=50, title='Distribución de las Puntuaciones de Perfumes',
                   color_discrete_sequence=['#636EFA'])
fig.update_layout(xaxis_title='Puntuación', yaxis_title='Recuento', bargap=0.1)
fig.show()

In [8]:
# Histograma del Conteo de Puntuaciones (Escala Logarítmica para mejor visibilidad)
fig = px.histogram(df, x='Rating Count', nbins=100, log_y=True, title='Distribución del Número de Puntuaciones (Escala Log)',
                   color_discrete_sequence=['#EF553B'])
fig.update_layout(xaxis_title='Número de Puntuaciones', yaxis_title='Recuento (Log)', bargap=0.1)
fig.show()

### 🏆 2.2 Marcas Principales

In [9]:
# Top 20 Marcas por número de perfumes
top_brands = df['Brand'].value_counts().head(20).reset_index()
top_brands.columns = ['Brand', 'Count']

fig = px.bar(top_brands, x='Count', y='Brand', orientation='h', title='Top 20 Marcas por Número de Perfumes',
             color='Count', color_continuous_scale='Viridis')
fig.update_layout(yaxis={'categoryorder':'total ascending'})
fig.show()

### 🚻 2.3 Distribución por Género

In [10]:
# Distribución de Género
gender_counts = df['Gender'].value_counts().reset_index()
gender_counts.columns = ['Gender', 'Count']

fig = px.pie(gender_counts, values='Count', names='Gender', title='Distribución de Perfumes por Género',
             color_discrete_sequence=px.colors.qualitative.Pastel)
fig.update_traces(textposition='inside', textinfo='percent+label')
fig.show()

### 📅 2.4 Distribución por Año de Lanzamiento

In [11]:
# Limpiar columna Año (convertir a numérico, forzar errores)
df['Year_Clean'] = pd.to_numeric(df['Year'], errors='coerce')

# Filtrar años poco realistas (aunque existen perfumes antiguos)
# Comprobemos el rango primero
print(df['Year_Clean'].describe())

# Plot distribución de Años de Lanzamiento
fig = px.histogram(df, x='Year_Clean', nbins=100, title='Distribución de Años de Lanzamiento',
                   color_discrete_sequence=['#00CC96'])
fig.update_layout(xaxis_title='Año', yaxis_title='Recuento', bargap=0.1)
fig.show()

count    22026.000000
mean      2012.455961
std         13.526737
min       1781.000000
25%       2010.000000
50%       2015.000000
75%       2019.000000
max       2024.000000
Name: Year_Clean, dtype: float64


## 📈 3. Análisis Bivariante y Multivariante

### 📉 3.1 Puntuaciones vs. Año de Lanzamiento

In [12]:
# Gráfico de dispersión de Puntuaciones vs Año
# Agregamos por año para ver la tendencia de la puntuación media a lo largo del tiempo
year_stats = df.groupby('Year_Clean')['Rating Value'].agg(['mean', 'count']).reset_index()
year_stats = year_stats[year_stats['count'] > 10] # Filtrar años con pocos perfumes para estabilidad

fig = px.scatter(year_stats, x='Year_Clean', y='mean', size='count', title='Puntuación Media por Año de Lanzamiento',
                 labels={'Year_Clean': 'Año', 'mean': 'Puntuación Media', 'count': 'Número de Perfumes'},
                 color='mean', color_continuous_scale='RdBu')
fig.show()

### 🍋 3.2 Análisis de Notas Populares

In [13]:
def get_top_notes(df, column, top_n=20):
    # Dividir cadenas por coma (usando regex para manejar espacios opcionales)
    notes = df[column].dropna().astype(str).str.split(r',\s*').explode()
    # Eliminar espacios en blanco y convertir a minúsculas
    notes = notes.str.strip().str.lower()
    # Eliminar cadenas vacías
    notes = notes[notes != '']
    return notes.value_counts().head(top_n)

# Analizar Notas de Salida (Top), Corazón (Middle) y Fondo (Base)
top_notes = get_top_notes(df, 'Top')
middle_notes = get_top_notes(df, 'Middle')
base_notes = get_top_notes(df, 'Base')

# Imprimir las notas más populares para verificación
print("Top 5 Notas de Salida:")
print(top_notes.head(5))

# Gráfico Notas de Salida
fig = px.bar(x=top_notes.values, y=top_notes.index, orientation='h', title='Notas de Salida Más Populares',
             labels={'x': 'Recuento', 'y': 'Nota'}, color=top_notes.values, color_continuous_scale='Blues')
fig.update_layout(yaxis={'categoryorder':'total ascending', 'automargin': True})
fig.show()

Top 5 Notas de Salida:
Top
bergamot           8535
mandarin orange    3933
lemon              2966
grapefruit         2185
pink pepper        2016
Name: count, dtype: int64


In [14]:
# Gráfico Notas de Fondo (a menudo definen el carácter del perfume)
fig = px.bar(x=base_notes.values, y=base_notes.index, orientation='h', title='Notas de Fondo Más Populares',
             labels={'x': 'Recuento', 'y': 'Nota'}, color=base_notes.values, color_continuous_scale='Reds')
fig.update_layout(yaxis={'categoryorder':'total ascending', 'automargin': True})
fig.show()

### 👫 3.3 Preferencias de Notas por Género

In [15]:
# Comparar Notas de Salida para Hombres vs Mujeres
men_df = df[df['Gender'] == 'men']
women_df = df[df['Gender'] == 'women']

men_notes = get_top_notes(men_df, 'Top', 10)
women_notes = get_top_notes(women_df, 'Top', 10)

fig = make_subplots(rows=1, cols=2, subplot_titles=('Notas de Salida en Perfumes de Hombre', 'Notas de Salida en Perfumes de Mujer'))

fig.add_trace(go.Bar(x=men_notes.values, y=men_notes.index, orientation='h', name='Hombres', marker_color='blue'), row=1, col=1)
fig.add_trace(go.Bar(x=women_notes.values, y=women_notes.index, orientation='h', name='Mujeres', marker_color='pink'), row=1, col=2)

fig.update_layout(title_text='Comparación por Género: Preferencia de Notas de Salida', showlegend=False)
fig.show()

### 🔢 3.4 Análisis de Correlación

In [16]:
# Matriz de Correlación de variables numéricas
numeric_df = df[['Rating Value', 'Rating Count', 'Year_Clean']]
corr = numeric_df.corr()

fig = px.imshow(corr, text_auto=True, title='Matriz de Correlación',
                color_continuous_scale='RdBu_r', aspect='auto')
fig.show()

### 🌍 3.5 Análisis por País

In [17]:
# Puntuación Media por País (Top 15 países productores)
top_countries = df['Country'].value_counts().head(15).index
country_stats = df[df['Country'].isin(top_countries)].groupby('Country')['Rating Value'].mean().sort_values(ascending=False).reset_index()

fig = px.bar(country_stats, x='Rating Value', y='Country', orientation='h', title='Puntuación Media por País (Top 15 Productores)',
             color='Rating Value', color_continuous_scale='Magma')
fig.update_layout(yaxis={'categoryorder':'total ascending'}, xaxis_range=[3.5, 4.5]) # Zoom en el rango de puntuación
fig.show()

## ✨ 4. Visualizaciones Avanzadas

### ☀️ 4.1 Gráfico Sunburst: Género y Acordes Principales

In [18]:
# Gráfico Sunburst mostrando jerarquía: Género -> Acorde Principal 1 -> Acorde Principal 2
# Necesitamos agrupar datos para esto
sunburst_data = df.groupby(['Gender', 'mainaccord1', 'mainaccord2']).size().reset_index(name='count')
# Filtrar para mejor visibilidad (eliminar grupos pequeños)
sunburst_data = sunburst_data[sunburst_data['count'] > 50]

fig = px.sunburst(sunburst_data, path=['Gender', 'mainaccord1', 'mainaccord2'], values='count',
                  title='Distribución de Acordes por Género',
                  color='count', color_continuous_scale='RdBu')
fig.show()

### 🔵 4.2 Scatter Interactivo: Rendimiento de Marcas

In [19]:
# Gráfico de dispersión de Top 50 Marcas: Puntuación Media vs Total de Puntuaciones
top_50_brands = df['Brand'].value_counts().head(50).index
brand_stats = df[df['Brand'].isin(top_50_brands)].groupby('Brand').agg(
    Avg_Rating=('Rating Value', 'mean'),
    Total_Ratings=('Rating Count', 'sum'),
    Perfume_Count=('Perfume', 'count')
).reset_index()

fig = px.scatter(brand_stats, x='Total_Ratings', y='Avg_Rating', size='Perfume_Count', color='Brand',
                 hover_name='Brand', log_x=True, title='Rendimiento de Marca: Puntuación vs Popularidad (Top 50)',
                 labels={'Total_Ratings': 'Total Puntuaciones (Log)', 'Avg_Rating': 'Puntuación Media', 'Perfume_Count': 'Número de Perfumes'})
fig.show()

### 🕸️ 4.3 Perfil Olfativo de Marca (Gráfico de Radar)

In [20]:
# Comparar perfiles olfativos de 3 grandes marcas: Chanel, Dior, Tom Ford
target_brands = ['chanel', 'dior', 'tom-ford']
radar_data = []

for brand in target_brands:
    brand_df = df[df['Brand'] == brand]
    # Obtener top 5 acordes para esta marca
    top_accords = brand_df['mainaccord1'].value_counts(normalize=True).head(5)
    for accord, prop in top_accords.items():
        radar_data.append({'Brand': brand, 'Accord': accord, 'Proportion': prop})

radar_df = pd.DataFrame(radar_data)

# Como diferentes marcas tienen diferentes acordes top, un radar estándar es complicado.
# Elijamos un conjunto de acordes comunes para compararlas.
common_accords = ['floral', 'woody', 'citrus', 'amber', 'spicy', 'fruity', 'aromatic', 'musky']

comparison_data = []
for brand in target_brands:
    brand_df = df[df['Brand'] == brand]
    total = len(brand_df)
    for accord in common_accords:
        count = brand_df[brand_df['mainaccord1'].str.contains(accord, case=False, na=False)].shape[0]
        comparison_data.append({'Brand': brand, 'Accord': accord, 'Percentage': (count/total)*100})

comp_df = pd.DataFrame(comparison_data)

fig = px.line_polar(comp_df, r='Percentage', theta='Accord', color='Brand', line_close=True,
                    title='Comparación de Perfil Olfativo: Chanel vs Dior vs Tom Ford',
                    markers=True)
fig.update_traces(fill='toself')
fig.show()

## 🤖 5. Sistema de Recomendación

Vamos a construir un sistema de recomendación basado en contenido. La idea es recomendar perfumes que tengan notas y acordes similares a uno dado.

### Pasos:
1.  **Preprocesamiento**: Combinar todas las notas (Salida, Corazón, Fondo) y los acordes principales en una sola cadena de texto para cada perfume.
2.  **Vectorización**: Usar TF-IDF (Term Frequency-Inverse Document Frequency) para convertir estas cadenas de texto en vectores numéricos.
3.  **Similitud**: Calcular la similitud del coseno entre estos vectores para encontrar los perfumes más cercanos.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# 1. Preprocesamiento: Crear una 'sopa' de características
# Limpiar nombres de perfumes (quitar guiones y poner mayúsculas)
df['Perfume'] = df['Perfume'].str.replace('-', ' ').str.title()

# Rellenar nulos con cadenas vacías
df['Top'] = df['Top'].fillna('')
df['Middle'] = df['Middle'].fillna('')
df['Base'] = df['Base'].fillna('')
df['mainaccord1'] = df['mainaccord1'].fillna('')
df['mainaccord2'] = df['mainaccord2'].fillna('')
df['mainaccord3'] = df['mainaccord3'].fillna('')

# Combinar columnas
df['soup'] = df['Top'] + ' ' + df['Middle'] + ' ' + df['Base'] + ' ' + \
             df['mainaccord1'] + ' ' + df['mainaccord2'] + ' ' + df['mainaccord3']

# 2. Vectorización TF-IDF
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['soup'])

print(f"Dimensiones de la matriz TF-IDF: {tfidf_matrix.shape}")

Dimensiones de la matriz TF-IDF: (24063, 1259)


In [22]:
# 3. Calcular Similitud del Coseno
# Nota: Para datasets muy grandes, esto puede consumir mucha memoria.
# Aquí calcularemos la similitud bajo demanda o para un subconjunto si fuera necesario.
# Dado que tenemos ~24k perfumes, la matriz de similitud sería 24k x 24k (~576M elementos), lo cual es manejable en memoria moderna (unos pocos GB).

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# Crear un mapeo inverso de índices y nombres de perfumes
indices = pd.Series(df.index, index=df['Perfume']).drop_duplicates()

In [23]:
def get_recommendations(title, cosine_sim=cosine_sim):
    try:
        # Obtener el índice del perfume
        idx = indices[title]

        # Si hay duplicados, tomar el primero
        if isinstance(idx, pd.Series):
            idx = idx.iloc[0]

        # Obtener las puntuaciones de similitud de todos los perfumes con ese perfume
        sim_scores = list(enumerate(cosine_sim[idx]))

        # Ordenar los perfumes según las puntuaciones de similitud
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

        # Obtener las puntuaciones de los 10 perfumes más similares (excluyendo el mismo)
        sim_scores = sim_scores[1:11]

        # Obtener los índices de los perfumes
        perfume_indices = [i[0] for i in sim_scores]

        # Devolver los 10 perfumes más similares
        return df[['Perfume', 'Brand', 'mainaccord1', 'mainaccord2']].iloc[perfume_indices]
    except KeyError:
        return "Perfume no encontrado en la base de datos."

In [35]:
# Probar el sistema de recomendación
# Ejemplo: Buscar recomendaciones para un perfume popular, e.g., 'Acqua di Gio'
# Primero busquemos el nombre exacto en el dataset
search_term = 'Althair'
matches = df[df['Perfume'].str.contains(search_term, case=False, na=False)]['Perfume'].head(5).tolist()
print(f"Posibles coincidencias para '{search_term}': {matches}")

if matches:
    example_perfume = matches[0]
    print(f"\nRecomendaciones para '{example_perfume}':")
    display(get_recommendations(example_perfume))

Posibles coincidencias para 'Althair': ['Althair']

Recomendaciones para 'Althair':


,Perfume,Brand,mainaccord1,mainaccord2
24044,Liquid Brun,fragrance-world,sweet,vanilla
22096,Mercedes Benz Select Night,mercedes-benz,woody,warm spicy
21501,Meharees,l-erbolario,woody,warm spicy
23375,Spice Black Vanilla,cremo,vanilla,warm spicy
16035,58 Avenue Montaigne Pour Homme Limited Edition,s-t-dupont,warm spicy,amber
19873,Fetes Persanes,mdci-parfums,warm spicy,woody
15335,Moonlight Gypsy,pinrose,woody,sweet
5017,Apres,ellis-brooklyn,woody,warm spicy
12360,Man Gold,zara,warm spicy,citrus
22119,Selection,jacques-battini,warm spicy,woody


## 💡 6. Insights y Conclusión

En este análisis, hemos explorado el dataset de Fragantica para comprender el panorama de los perfumes. Hemos analizado puntuaciones, marcas, preferencias de género y notas olfativas.

### Conclusiones Clave:

1.  **Distribución de Puntuaciones**: Las puntuaciones de los perfumes tienden a estar sesgadas hacia el extremo superior (3.5 - 4.5), lo que sugiere que los usuarios generalmente califican los perfumes que les gustan o que la calidad es generalmente alta.
2.  **Dominio de Marca**: Unas pocas marcas grandes dominan el mercado en términos de número de perfumes lanzados. Sin embargo, las marcas nicho a menudo logran puntuaciones medias altas.
3.  **Preferencias de Género**: Existen diferencias claras en las preferencias de notas entre géneros. Las notas florales y frutales son más frecuentes en los perfumes de mujer, mientras que las notas amaderadas y aromáticas son comunes en los perfumes de hombre.
4.  **Evolución en el Tiempo**: El número de lanzamientos de perfumes ha aumentado significativamente en los últimos años.
5.  **Perfiles Olfativos**: Diferentes marcas tienen firmas olfativas distintas, como se ve en la comparación del gráfico de radar.